In [45]:
from utils.news import get_news, get_url_content
from utils.price import get_price
from utils.llms import summarize, get_trading_recommendation
import pandas as pd
from tqdm import tqdm

In [56]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

def generate_summary(stock, title, content):

    llm = Ollama(model="llama3")

    summary_template = """
    You are an AI assistant specialized in investment advisory. 
    Provide a concise summary of the news article focusing only on the specified stock, interpret the investment sentiment, and give a score between -1 to 1 (where -1 indicates 'sell' and 1 indicates 'buy'):

    Stock: {stock}

    Title: {title}

    Content: {content}

    Summary:
    Sentiment Score: 
    """


    summary_prompt = PromptTemplate(template=summary_template, input_variables=["stock", "title", "content"])

    # Create the chain
    chain = LLMChain(llm=llm, prompt=summary_prompt)

    # Run the chain to get the summary
    summary = chain.run(stock = stock, title=title, content=content)
    
    return summary

In [87]:
def generate_stock_price_impact(stock, title, content):

    llm = Ollama(model="llama3")

    stock_price_impact_template = """
    You are an AI assistant specialized in investment advisory. 
    Provide a concise summary of the news article focusing only on the specified stock, interpret the expected immediate impact of this news on the stock price as a score between -1 and 1 (where -1 indicates a negative impact and 1 indicates a positive impact):

    Stock: {stock}

    Title: {title}

    Content: {content}
    Summary:
    Impact Score:
    """

    stock_price_impact_prompt = PromptTemplate(template=stock_price_impact_template, input_variables=["stock", "title", "content"])

    # Create the chain
    chain = LLMChain(llm=llm, prompt=stock_price_impact_prompt)

    # Run the chain to get the impact score on stock price
    impact_score = chain.run(stock=stock, title=title, content=content)
    
    return impact_score

In [66]:
def generate_sentiment_score(stock, title, content):

    llm = Ollama(model="llama3")

    sentiment_score_template = """
    You are an AI assistant specialized in investment advisory. 
    Analyze the news article focusing only on the specified stock and provide only the sentiment score between -1 to 1 (where -1 indicates 'sell' and 1 indicates 'buy'):

    Stock: {stock}

    Title: {title}

    Content: {content}

    Sentiment Score:
    """

    sentiment_score_prompt = PromptTemplate(template=sentiment_score_template, input_variables=["stock", "title", "content"])

    # Create the chain
    chain = LLMChain(llm=llm, prompt=sentiment_score_prompt)

    # Run the chain to get the sentiment score
    sentiment_score = chain.run(stock=stock, title=title, content=content)
    
    return sentiment_score

In [57]:
topic = news_df.topic.values[0]
content = news_df.content.values[0]

In [89]:
impact = generate_stock_price_impact(stock, topic, content)

In [90]:
print(impact)

Summary: The article discusses Nvidia's performance in early 2024, highlighting a significant 96% bull rally followed by a 13% dip. The company is currently in a consolidation phase, with its stock price oscillating between $756 and $974. Analysts expect the first-quarter earnings report on May 22 to be positive, with forecasts of $5.18 per share.

Impact Score: +0.7

The expected immediate impact of this news on Nvidia's stock price is neutral-positive. The article highlights the company's solid performance in early 2024 and analysts' optimistic forecast for its first-quarter earnings report. However, the consolidation phase and the potential for a dip below $756 if the earnings report does not meet expectations temper the enthusiasm. Overall, the news suggests that Nvidia is poised for growth, but investors should be cautious and monitor key support and resistance levels closely.


In [95]:
pattern = r"Impact Score: ([+-]?\d+(\.\d+)?)"

In [96]:
import re
match = re.search(pattern, impact)

In [97]:
if match:
    impact = float(match.group(1))

In [98]:
impact

0.7

In [2]:
stock = "Nvidia"
symbol = "NVDA"  
date = "2024-05-02"
news_duration = 1
price_duration = 7
sector_info='''Nvidia operates in the semiconductor industry, specializing in the design and manufacture of graphics processing units (GPUs) for gaming, professional visualization, data centers, AI, and automotive applications. 
Known for its GeForce brand, Nvidia leads the gaming market while also providing high-performance solutions for AI and machine learning in data centers. 
Additionally, Nvidia's DRIVE platform supports autonomous vehicle technology, and its Jetson platform powers edge computing and IoT devices. 
The company's innovations position it at the forefront of emerging trends in AI, data processing, and autonomous driving, driving significant growth and industry influence.
'''

In [4]:
news_df = get_news(symbol, date, news_duration)

In [9]:
headlines = news_df.headline.values.tolist()

In [38]:
topic_list=[]
content_list=[]
for url in tqdm(news_df['url'].values.tolist()):
    topic, content = get_url_content(url)    
    topic_list.append(topic)
    content_list.append(content)    

In [40]:
news_df['topic'] = topic_list
news_df['content'] = content_list

In [ ]:
impact_summary_list=[]
for i in tqdm(range(len(news_df))):
    topic = news_df.topic.values[i]
    content = news_df.content.values[i]
    summary = summarize(stock, topic, content)
    summary_list.append(summary)

In [47]:
summary_list=[]
for i in tqdm(range(len(news_df))):
    topic = news_df.topic.values[i]
    content = news_df.content.values[i]
    summary = summarize(stock, topic, content)
    summary_list.append(summary)

100%|███████████████████████████████████████████| 13/13 [03:01<00:00, 13.94s/it]


In [48]:
news_df['summary'] = summary_list

In [49]:
news_df['summary'].values

array(["Summary:\n\nNvidia's stock has experienced a rollercoaster ride in early 2024, with a 96% bull rally followed by a 13% dip. The company is currently in a consolidation phase, oscillating between $756 and $974. Analysts are forecasting earnings of $5.18 per share for the first quarter, which could impact the stock's performance.\n\nInterpretation:\n\nThe investment sentiment surrounding Nvidia (NVDA) appears to be cautious but optimistic. The recent consolidation phase suggests that investors are taking a wait-and-see approach, waiting for the company's upcoming first-quarter earnings report on May 22. A positive earnings report could push the stock back above its previous highs, while a negative report or disappointing results might lead to a downturn below $756.\n\nKey takeaways:\n\n1. Nvidia's stock is currently in a consolidation phase, reflecting investor uncertainty.\n2. The company's upcoming first-quarter earnings report on May 22 will be crucial for determining the next

In [50]:
results = nlp(summary_list)

In [51]:
results

[{'label': 'Negative', 'score': 0.9275181889533997},
 {'label': 'Positive', 'score': 0.9999991655349731},
 {'label': 'Positive', 'score': 0.9999995231628418},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Neutral', 'score': 0.9435479044914246},
 {'label': 'Negative', 'score': 0.9999998807907104},
 {'label': 'Positive', 'score': 0.9999364614486694},
 {'label': 'Positive', 'score': 0.9998644590377808},
 {'label': 'Negative', 'score': 0.9983217120170593},
 {'label': 'Neutral', 'score': 0.9998190999031067},
 {'label': 'Positive', 'score': 1.0},
 {'label': 'Negative', 'score': 0.9999513626098633},
 {'label': 'Positive', 'score': 0.9940310120582581}]

In [14]:
headlines

["Nvidia's Roller Coaster Ride Through Early 2024 Sets The Stage For A Potential Market Shakeup As Investors Eye The Critical First Quarter Earnings Report",
 'Super Micro Updates Fiscal Guidance As Nvidia Shifts, 4 Analysts Provide Perspectives',
 "Intel Gains Favor With Redditors Over Nvidia, AMD As 'High Uncertainty, Low Risk' Stock Play",
 "Nvidia Backed AI Start-Up CoreWeave Lands New $1.1B Funding - What's On The Cards?",
 "10 Information Technology Stocks With Whale Alerts In Today's Session",
 "US Stocks Mixed Ahead Of Fed Meeting, Chipmakers Tumble After AMD Earnings, Bitcoin Sinks: What's Driving Markets Wednesday?",
 "AMD Falls On Q1 Earnings, Q2 Guidance: Why '1,000% Growth' From AI Stock Is Not Enough For Investors",
 'After Nvidia and Apple, Alibaba Chases Vietnam: New Data Center to Boost Control and Meet Local Laws',
 'Super Micro Folly, Stock Market Direction Depends On Which Powell Shows Up – Objective Or Political',
 'Market Clubhouse Morning Memo - May 1st, 2024 (Tr

In [15]:
news_df.summary.values.tolist()

['\n\tNvidia started 2024 with a strong performance, witnessing a 96% bull rally.\n\tSurpassing $974 might indicate further growth.\n\tAttention is turning towards Nvidia’s anticipated first-quarter earnings report.\n\n\n',
 'Wedbush analyst maintains Super Micro Computer Inc with Neutral rating and raises price target from $530 to $800, citing sales momentum and concerns about margins and competition in AI server market.',
 'Intel is attracting attention from Reddit&#39;s ValueInvesting community for its potential turnaround and long-term growth. With a favorable market cap compared to competitors and investments in cutting-edge technologies, investors see opportunity for substantial returns.',
 'CoreWeave, a cloud-computing firm backed by\xa0NVIDIA, has\xa0secured\xa0$1.1 billion\xa0in new funding, led by Coatue, with participation from Magnetar, who led the last primary round, as well as Altimeter Capital, Fidelity Management &amp; Research Company, and Lykos Global Management.',
 '

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [10]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [11]:
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [99]:
from utils.finbert import get_sentiment

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [100]:
summary=summary_list[0]

In [101]:
sentiment = get_sentiment(summary)

In [105]:
sentiment[0]['label']

'Negative'

In [102]:
results = nlp(headlines)

In [24]:
label = []
score = []
for i in range(len(results)):
    label.append(results[i]['label'])
    score.append(results[i]['score'])    

In [25]:
news_df['label'] = label
news_df['score'] = score 

In [26]:
news_df

,id,author,created_at,updated_at,headline,summary,source,url,symbols,label,score
0,38569426,Zaheer Anwari,2024-05-01T22:35:07Z,2024-05-01T22:35:49Z,Nvidia's Roller Coaster Ride Through Early 202...,\n\tNvidia started 2024 with a strong performa...,benzinga,https://www.benzinga.com/trading-ideas/24/05/3...,[NVDA],Positive,0.935510
1,38563406,Anusuya Lahiri,2024-05-01T19:44:09Z,2024-05-01T19:44:09Z,Super Micro Updates Fiscal Guidance As Nvidia ...,Wedbush analyst maintains Super Micro Computer...,benzinga,https://www.benzinga.com/news/24/05/38563406/s...,"[IDGT, NVDA, SMCI, XMMO]",Neutral,0.999987
2,38563400,Surbhi Jain,2024-05-01T19:43:46Z,2024-05-01T19:43:47Z,"Intel Gains Favor With Redditors Over Nvidia, ...",Intel is attracting attention from Reddit&#39;...,benzinga,https://www.benzinga.com/trading-ideas/long-id...,"[AMD, INTC, NVDA, TSM]",Positive,0.999997
3,38561494,Shivani Kumaresan,2024-05-01T18:16:13Z,2024-05-01T18:16:14Z,Nvidia Backed AI Start-Up CoreWeave Lands New ...,"CoreWeave, a cloud-computing firm backed by NV...",benzinga,https://www.benzinga.com/markets/equities/24/0...,"[AMZN, MSFT, NVDA, NVDX, USD]",Neutral,0.999580
4,38560566,Benzinga Insights,2024-05-01T17:35:13Z,2024-05-01T17:35:14Z,10 Information Technology Stocks With Whale Al...,,benzinga,https://www.benzinga.com/insights/options/24/0...,"[AAPL, AMD, FSLY, MRVL, MU, NVDA, SMCI, SWKS, ...",Neutral,0.999794
5,38559813,Piero Cingari,2024-05-01T17:07:02Z,2024-05-01T17:07:03Z,"US Stocks Mixed Ahead Of Fed Meeting, Chipmake...",Traders are treading cautiously in Wednesday’s...,benzinga,https://www.benzinga.com/markets/equities/24/0...,"[ALB, ALL, AMD, AMZN, BTCUSD, BZH, CHRW, DIA, ...",Negative,0.999287
6,38558952,Chris Katje,2024-05-01T16:39:17Z,2024-05-01T16:39:18Z,"AMD Falls On Q1 Earnings, Q2 Guidance: Why '1,...","AMD raised its guidance for AI segment growth,...",benzinga,https://www.benzinga.com/analyst-ratings/analy...,"[AMD, NVDA]",Negative,0.824331
7,38556060,Anusuya Lahiri,2024-05-01T15:21:22Z,2024-05-01T15:21:23Z,"After Nvidia and Apple, Alibaba Chases Vietnam...",Alibaba to build data center in Vietnam to com...,benzinga,https://www.benzinga.com/government/24/05/3855...,"[AAPL, AMZN, BABA, CGRO, GOOG, GOOGL, LRCX, NV...",Neutral,0.804862
8,38554907,The Arora Report,2024-05-01T14:57:45Z,2024-05-01T14:59:19Z,"Super Micro Folly, Stock Market Direction Depe...","To gain an edge, this is what you need to know...",benzinga,https://www.benzinga.com/markets/24/05/3855490...,"[AAPL, ADP, AMZN, BTCUSD, GLD, GOOG, META, MSF...",Neutral,0.999942
9,38553582,RIPS,2024-05-01T14:23:03Z,2024-05-01T14:23:34Z,"Market Clubhouse Morning Memo - May 1st, 2024 ...",Good Morning Traders! In today&#39;s Market Cl...,benzinga,https://www.benzinga.com/markets/24/05/3855358...,"[AAPL, GOOGL, META, MSFT, NVDA, QQQ, SPY, TSLA]",Neutral,0.999996


In [ ]:
url = df['url'].values[0]